In [1]:
import numpy as np, pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# ==== DỮ LIỆU GỐC ====
R = pd.DataFrame(
    [[4,3,np.nan,4],
     [3,3,5,5],
     [5,4,3,4],
     [np.nan,2,3,5]],
    index=['U1','U2','U3','U4'], columns=['I1','I2','I3','I4']
)
R


,I1,I2,I3,I4
U1,4.0,3,NaN,4
U2,3.0,3,5.0,5
U3,5.0,4,3.0,4
U4,NaN,2,3.0,5


## 1.User–User Pearson Similarity (theo đề gốc)
- Dự đoán r<sub>U1,I3</sub> và r<sub>U4,I1</sub>

In [2]:
def pearson(u, v):
    a, b = R.loc[u], R.loc[v]
    mask = a.notna() & b.notna()
    if mask.sum() < 2: return 0.0
    au, bv = a[mask] - a[mask].mean(), b[mask] - b[mask].mean()
    num = (au * bv).sum()
    den = np.sqrt((au**2).sum() * (bv**2).sum())
    return 0.0 if den == 0 else num / den

def predict_pearson(u, i):
    mu_u = R.loc[u].mean()
    num = den = 0
    for v in R.index:
        if v == u or pd.isna(R.loc[v, i]): continue
        s = pearson(u, v)
        mu_v = R.loc[v].mean()
        num += s * (R.loc[v, i] - mu_v)
        den += abs(s)
    return mu_u if den == 0 else np.clip(mu_u + num / den, 1, 5)

print("🎯 Pearson:")
print("r_U1,I3 =", round(predict_pearson('U1', 'I3'), 3))
print("r_U4,I1 =", round(predict_pearson('U4', 'I1'), 3))


🎯 Pearson:
r_U1,I3 = 3.5
r_U4,I1 = 3.213


## 2.Centered-Cosine Similarity (chuẩn hoá mean)
- Trừ trung bình mỗi user → cosine similarity

In [3]:
Rc = R.sub(R.mean(axis=1), axis=0).fillna(0.0)
S_cos = pd.DataFrame(cosine_similarity(Rc), index=R.index, columns=R.index)

def predict_centered_cosine(u, i):
    mu_u = R.loc[u].mean()
    num = den = 0
    for v in R.index:
        if v == u or pd.isna(R.loc[v, i]): continue
        s = S_cos.loc[u, v]
        num += s * (R.loc[v, i] - R.loc[v].mean())
        den += abs(s)
    return mu_u if den == 0 else np.clip(mu_u + num / den, 1, 5)

print("\n🧮 Centered-Cosine:")
print("r_U1,I3 =", round(predict_centered_cosine('U1','I3'),3))
print("r_U4,I1 =", round(predict_centered_cosine('U4','I1'),3))



🧮 Centered-Cosine:
r_U1,I3 = 3.565
r_U4,I1 = 3.181


## 3.Item–Item Pearson (biến thể đề có thể ra)
- Dự đoán dựa trên tương đồng giữa item I<sub>3</sub> & các item user U1 đã rate.

In [4]:
def pearson_item(i, j):
    a, b = R[i], R[j]
    mask = a.notna() & b.notna()
    if mask.sum() < 2: return 0.0
    ai, bj = a[mask] - a[mask].mean(), b[mask] - b[mask].mean()
    num = (ai * bj).sum()
    den = np.sqrt((ai**2).sum() * (bj**2).sum())
    return 0.0 if den == 0 else num / den

def predict_item_based(u, i):
    mu_i = R[i].mean()
    num = den = 0
    for j in R.columns:
        if j == i or pd.isna(R.loc[u, j]): continue
        s = pearson_item(i, j)
        mu_j = R[j].mean()
        num += s * (R.loc[u, j] - mu_j)
        den += abs(s)
    return mu_i if den == 0 else np.clip(mu_i + num / den, 1, 5)

print("\n🔹 Item-Item Pearson:")
print("r_U1,I3 =", round(predict_item_based('U1','I3'),3))
print("r_U4,I1 =", round(predict_item_based('U4','I1'),3))



🔹 Item-Item Pearson:
r_U1,I3 = 3.5
r_U4,I1 = 3.769


## 4.Z-Score Normalization (biến thể đề nâng cao)
- Chuẩn hoá mỗi user về Z-score, tính Pearson, giải chuẩn hoá ngược.

In [5]:
def predict_zscore(u,i):
    zR = R.apply(lambda x: (x - x.mean()) / x.std(), axis=1)
    def pearson_z(u,v):
        a,b = zR.loc[u], zR.loc[v]
        mask = a.notna() & b.notna()
        if mask.sum()<2: return 0
        num = ((a[mask]-a[mask].mean())*(b[mask]-b[mask].mean())).sum()
        den = np.sqrt(((a[mask]-a[mask].mean())**2).sum()*((b[mask]-b[mask].mean())**2).sum())
        return 0 if den==0 else num/den
    mu_u, sigma_u = R.loc[u].mean(), R.loc[u].std()
    num = den = 0
    for v in R.index:
        if v==u or pd.isna(R.loc[v,i]): continue
        s = pearson_z(u,v)
        num += s * ((R.loc[v,i] - R.loc[v].mean())/R.loc[v].std())
        den += abs(s)
    return mu_u if den==0 else np.clip(mu_u + sigma_u*(num/den),1,5)

print("\n📊 Z-score:")
print("r_U1,I3 =", round(predict_zscore('U1','I3'),3))
print("r_U4,I1 =", round(predict_zscore('U4','I1'),3))



📊 Z-score:
r_U1,I3 = 3.552
r_U4,I1 = 3.454


## 5.Baseline (Global mean + user bias + item bias)

In [6]:
# Global mean
mu = R.stack().mean()
# User bias & item bias
bu = R.mean(axis=1) - mu
bi = R.mean(axis=0) - mu

def predict_baseline(u,i):
    base = mu + bu[u] + bi[i]
    return np.clip(base,1,5)

print("\n📈 Baseline bias model:")
print("r_U1,I3 =", round(predict_baseline('U1','I3'),3))
print("r_U4,I1 =", round(predict_baseline('U4','I1'),3))



📈 Baseline bias model:
r_U1,I3 = 3.548
r_U4,I1 = 3.548


## 6.Slope-One (biến thể hay ra đề “tính chênh lệch trung bình”)

In [7]:
def slope_one(u,i):
    diffs, counts = {}, {}
    for j in R.columns:
        if j==i: continue
        mask = R[i].notna() & R[j].notna()
        if mask.sum()<1: continue
        diffs[j] = (R.loc[mask,i] - R.loc[mask,j]).mean()
        counts[j] = mask.sum()
    num = den = 0
    for j in R.columns:
        if pd.isna(R.loc[u,j]) or j not in diffs: continue
        num += (R.loc[u,j] + diffs[j]) * counts[j]
        den += counts[j]
    return np.clip(num/den,1,5) if den>0 else np.nan

print("\n⚗️ Slope-One:")
print("r_U1,I3 =", round(slope_one('U1','I3'),3))
print("r_U4,I1 =", round(slope_one('U4','I1'),3))



⚗️ Slope-One:
r_U1,I3 = 3.5
r_U4,I1 = 3.5


## 6.Significance Weighting (giảm nhiễu khi giao nhau ít)

In [8]:
def slope_one(u,i):
    diffs, counts = {}, {}
    for j in R.columns:
        if j==i: continue
        mask = R[i].notna() & R[j].notna()
        if mask.sum()<1: continue
        diffs[j] = (R.loc[mask,i] - R.loc[mask,j]).mean()
        counts[j] = mask.sum()
    num = den = 0
    for j in R.columns:
        if pd.isna(R.loc[u,j]) or j not in diffs: continue
        num += (R.loc[u,j] + diffs[j]) * counts[j]
        den += counts[j]
    return np.clip(num/den,1,5) if den>0 else np.nan

print("\n⚗️ Slope-One:")
print("r_U1,I3 =", round(slope_one('U1','I3'),3))
print("r_U4,I1 =", round(slope_one('U4','I1'),3))



⚗️ Slope-One:
r_U1,I3 = 3.5
r_U4,I1 = 3.5


## 7.Significance Weighting (giảm nhiễu khi giao nhau ít)

In [9]:
def pearson_sig(u,v):
    a,b=R.loc[u],R.loc[v]
    mask=a.notna()&b.notna(); n=mask.sum()
    if n<2: return 0
    au,bv=a[mask]-a[mask].mean(),b[mask]-b[mask].mean()
    s=(au*bv).sum()/np.sqrt((au**2).sum()*(bv**2).sum())
    return s*(n/5)  # shrink về 0 nếu n<5

def predict_sig(u,i):
    mu_u=R.loc[u].mean(); num=den=0
    for v in R.index:
        if v==u or pd.isna(R.loc[v,i]): continue
        s=pearson_sig(u,v)
        mu_v=R.loc[v].mean()
        num+=s*(R.loc[v,i]-mu_v)
        den+=abs(s)
    return mu_u if den==0 else np.clip(mu_u+num/den,1,5)

print("\n🧮 Significance-weighted Pearson:")
print("r_U1,I3 =", round(predict_sig('U1','I3'),3))
print("r_U4,I1 =", round(predict_sig('U4','I1'),3))



🧮 Significance-weighted Pearson:
r_U1,I3 = 3.533
r_U4,I1 = 3.119


## 8.Tính toàn bộ ma trận dự đoán (cho các ô thiếu)

In [10]:
pred = R.copy()
for u in R.index:
    for i in R.columns:
        if pd.isna(R.loc[u,i]):
            pred.loc[u,i] = predict_pearson(u,i)
print("\n🗂️ Ma trận hoàn chỉnh dự đoán (Pearson):")
print(pred.round(3))



🗂️ Ma trận hoàn chỉnh dự đoán (Pearson):
       I1  I2   I3  I4
U1  4.000   3  3.5   4
U2  3.000   3  5.0   5
U3  5.000   4  3.0   4
U4  3.213   2  3.0   5
